<a href="https://colab.research.google.com/github/Charles1A/Nasdaq100_animated_scatter_plot/blob/main/Stock_animtd_scatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import scipy
import pandas as pd
import numpy as np
import plotly.express as px

import yfinance as yf

from datetime import datetime, timedelta, date
import time

In [2]:
print(f"pandas version: {pd.__version__}\n"\
f"numpy version: {np.__version__}\n"\
f"scipy version: {scipy.__version__}\n"\
f"yfinance version: {yf.__version__}\n")

print("plotly ")
!pip show plotly | grep Version

pandas version: 2.2.2
numpy version: 1.26.4
scipy version: 1.13.1
yfinance version: 0.2.50

plotly 
Version: 5.24.1


In [49]:
%%writefile requirements.txt

Writing requirements.txt


In [ ]:
# # Optional:
# # Load the Nasdaq 100 csv:

# df = pd.read_csv('/content/Nasdaq-100.csv')
# df.head(3)

In [ ]:
# df.info()

In [ ]:
# # Optional:
# # Create a list of tickers from the Nasdaq 100 csv:

# ticker_list = df.iloc[: ,0].to_list()

# print(ticker_list[0:5])

In [54]:
def fiscal_quarter_dates(year):
    """
    Computes the start and end dates of each fiscal quarter for the given year.

    Args:
      year: The year for which to compute the fiscal quarter dates.

    Returns:
      A list of tuples, where each tuple contains the start and end date of a fiscal quarter.
    """

    quarter_dates = []
    for quarter in range(1, 5):
        if quarter == 1:
            start_date = date(year - 1, 10, 1)
            end_date = date(year, 1, 1)
        elif quarter == 2:
            start_date = date(year, 1, 1)
            end_date = date(year, 4, 1)
        elif quarter == 3:
            start_date = date(year, 4, 1)
            end_date = date(year, 7, 1)
        elif quarter == 4:
            start_date = date(year, 7, 1)
            end_date = date(year + 1, 1, 1)
        quarter_dates.append((start_date, end_date))
    return quarter_dates

In [14]:
# current_year = date.today().year
# last_year_dates = fiscal_quarter_dates(current_year - 1)
# print(last_year_dates)

[(datetime.date(2022, 10, 1), datetime.date(2023, 1, 1)), (datetime.date(2023, 1, 1), datetime.date(2023, 4, 1)), (datetime.date(2023, 4, 1), datetime.date(2023, 7, 1)), (datetime.date(2023, 7, 1), datetime.date(2024, 1, 1))]


In [55]:
def last_4_fiscal_quarters():
    """
    Computes the start and end dates of the last 4 fiscal quarters.

    Returns:
        A list of tuples, where each tuple contains the start and end date of a fiscal quarter.
    """
    current_year = date.today().year
    last_year_dates = fiscal_quarter_dates(current_year - 1)  # Get last year's fiscal quarters
    current_year_dates = fiscal_quarter_dates(current_year)  # Get current year's fiscal quarters

    # Combine and return the last 4 quarters (Q4 of last year and Q1-Q3 of current year)
    return [last_year_dates[3]] + current_year_dates[:3]

In [56]:
fiscal_quarter_dates = last_4_fiscal_quarters()
print(fiscal_quarter_dates)

[(datetime.date(2023, 7, 1), datetime.date(2024, 1, 1)), (datetime.date(2023, 10, 1), datetime.date(2024, 1, 1)), (datetime.date(2024, 1, 1), datetime.date(2024, 4, 1)), (datetime.date(2024, 4, 1), datetime.date(2024, 7, 1))]


In [60]:
def yf_api_func(ticker_list, dates):

  df_list = []

  for i in dates:

    stock_df = yf.download(ticker_list, # Make call to the yfinance API
        start=i[0], end=i[1]) # returns the first and second entry in every ith tuple

    time.sleep(6) # 6-second pause between API calls;

    adj_close_df = stock_df.loc[:, 'Adj Close'].reset_index() # Isolate adjusted close values column & convert date index to column

    adj_close_df = adj_close_df.rename_axis(index=None, columns=None)

    adj_close_df['start date'] = i[0] # grabs the first entry in every ith tuple

    df_list.append(adj_close_df) # creates a list of dataframes

  full_df = pd.concat(df_list, ignore_index=True) # concatenates the list of dataframes into one dataframe

  return full_df

In [58]:
ticker_list = ['AMZN', 'TSLA', 'PLTR', 'GOOG', 'META', 'NVDA']

In [62]:
df_ = yf_api_func(ticker_list, fiscal_quarter_dates)

[*********************100%***********************]  6 of 6 completed
[*********************100%***********************]  6 of 6 completed
[*********************100%***********************]  6 of 6 completed
[*********************100%***********************]  6 of 6 completed


In [63]:
# Optional: Check df_
display(df_.head(3))
print("Number of rows, columns: ", df_.shape)

,Date,AMZN,GOOG,META,NVDA,PLTR,TSLA,start date
0,2023-07-03,130.220001,120.128365,285.161377,42.393757,15.52,279.820007,2023-07-01
1,2023-07-05,130.380005,122.190948,293.486328,42.297806,15.70,282.480011,2023-07-01
2,2023-07-06,128.360001,120.497040,291.113495,42.083900,15.13,276.540009,2023-07-01


Number of rows, columns:  (313, 8)


In [23]:
# # Optional: Save df_ to CSV for re-use

# df_.to_csv(path_or_buf='data.csv', index=False)

In [ ]:
# # Import csv to dataframe if necessary

# df_ = pd.read_csv('/content/')
# df_.head(3)

In [65]:
# Add a column that shows the quarter
df_['quarter'] = pd.to_datetime(df_['start date']).dt.to_period('Q')

In [66]:
# Optional: check df_ for modification
df_.head(3)

,Date,AMZN,GOOG,META,NVDA,PLTR,TSLA,start date,quarter
0,2023-07-03,130.220001,120.128365,285.161377,42.393757,15.52,279.820007,2023-07-01,2023Q3
1,2023-07-05,130.380005,122.190948,293.486328,42.297806,15.70,282.480011,2023-07-01,2023Q3
2,2023-07-06,128.360001,120.497040,291.113495,42.083900,15.13,276.540009,2023-07-01,2023Q3


In [67]:
# compute the pct return for each quarter

result = []
for i in df_['quarter'].unique():

  qtr_start = df_[df_['quarter'] == i].iloc[0,1:-2] # isolates the first row; excludes date columns
  qtr_end = df_[df_['quarter'] == i].iloc[-1,1:-2] # isolates the last row; excludes date columns

  qtr_return = ((qtr_end - qtr_start)/qtr_start).to_frame() *100 # Compute pct return for the quarter
  qtr_return['quarter'] = i
  qtr_return.columns = ['Pct Returns', 'quarter']

  result.append(qtr_return)

In [68]:
# Concatenate the list of dataframes in result:
qtr_returns_df = pd.concat(result, axis=0)

In [69]:
# Optional: check qtr_returns_df
qtr_returns_df.head(3)

,Pct Returns,quarter
AMZN,16.679466,2023Q3
GOOG,16.896148,2023Q3
META,23.753597,2023Q3


In [71]:
# compute daily returns within each quarter; result will be used to compute st dev

daily_result = []
for i in df_['quarter'].unique():

  qtr_daily = df_[df_['quarter'] == i].iloc[:,1:-2].pct_change() *100 # exclude date columns

  qtr_daily['quarter'] = i

  daily_result.append(qtr_daily)

In [72]:
# Concatenate the list of dataframes in daily_result:
qtr_daily_df = pd.concat(daily_result, axis=0)

In [73]:
# Optional: check qtr_daily_df
qtr_daily_df.head(2)

,AMZN,GOOG,META,NVDA,PLTR,TSLA,quarter
0,NaN,NaN,NaN,NaN,NaN,NaN,2023Q3
1,0.122872,1.716983,2.919382,-0.226333,1.15979,0.950612,2023Q3


In [76]:
# compute std dev (volatility) for each quarter;

stdev_result = []
for i in qtr_daily_df['quarter'].unique():

  qtr_stdev = qtr_daily_df[qtr_daily_df['quarter'] == i].iloc[:,:-1].std().to_frame()

  qtr_stdev['quarter'] = i

  qtr_stdev.reset_index(inplace=True)
  qtr_stdev.columns = ['Ticker', 'Std dev', 'quarter']

  stdev_result.append(qtr_stdev)

In [77]:
# Concatenate dataframes in stdev_result:
qtr_stdev_df = pd.concat(stdev_result, axis=0)

In [78]:
# Optional: check qtr_stdev_df
qtr_stdev_df.head(2)

,Ticker,Std dev,quarter
0,AMZN,1.927437,2023Q3
1,GOOG,1.738123,2023Q3


In [79]:
# Reset indices of both dataframes to preempt index error
qtr_returns_df = qtr_returns_df.reset_index(drop=True)
qtr_stdev_df = qtr_stdev_df.reset_index(drop=True)

In [85]:
# Concatenate the dataframes
concat_df = pd.concat([qtr_stdev_df, qtr_returns_df], axis=1)

In [86]:
# Optional: check concat_df
concat_df.head(3)

,Ticker,Std dev,quarter,Pct Returns,quarter
0,AMZN,1.927437,2023Q3,16.679466,2023Q3
1,GOOG,1.738123,2023Q3,16.896148,2023Q3
2,META,1.811973,2023Q3,23.753597,2023Q3


In [87]:
# Remove duplicate 'quarter' column from concat_df
concat_df = concat_df.loc[:,~concat_df.columns.duplicated()]
concat_df.head(3)

,Ticker,Std dev,quarter,Pct Returns
0,AMZN,1.927437,2023Q3,16.679466
1,GOOG,1.738123,2023Q3,16.896148
2,META,1.811973,2023Q3,23.753597


In [88]:
# Optional: Create a color category to highlight certain markers on the scatter plot
concat_df['Tracking'] = concat_df.iloc[: , 0].apply(lambda x: 'magenta' if x == 'TSLA' or x == 'NVDA' else 'steelblue')

In [92]:
# Optional: Check concat_df

concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype        
---  ------       --------------  -----        
 0   Ticker       24 non-null     object       
 1   Std dev      24 non-null     float64      
 2   quarter      24 non-null     period[Q-DEC]
 3   Pct Returns  24 non-null     object       
 4   Tracking     24 non-null     object       
dtypes: float64(1), object(3), period[Q-DEC](1)
memory usage: 1.1+ KB


In [93]:
# Create animated scatter plot via Plotly Express:

fig = px.scatter(concat_df,
                x='Std dev',
                y='Pct Returns',
                color='Tracking',
                hover_name='Ticker',
                 log_x=False,
                 animation_frame='quarter',
                 range_x=[0, 10],
                 range_y=[-60,120],
                 text='Ticker'
                )

fig.update_traces(textposition='top center')

fig.update_layout(
    showlegend=False,
    height=600,
    width=700,
    title_text='Stock volatility x return',
    title_x=0.5)

fig.update_traces(marker=dict(size=12,
                              line=dict(width=2, color='white')))

fig.add_hrect(y0=0, y1=-60,
              line_width=0,
              fillcolor="red", opacity=0.1,
                annotation_text="[NEGATIVE RETURN REGION]",
                annotation_position="top right",
                annotation_font_size=10,
                annotation_font_color="black")

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2750
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1500

fig.show()
fig.write_html("/content/stock-animtd-scatter-fig.html") # Save the output